In [4]:
import pandas as pd
from build_database import CocktailClient, SQLClient
import sqlite3

In [2]:
bar_data = pd.read_csv("data/bar_data.csv")
bar_data['glass_type'] = bar_data['glass_type'].replace('coper mug','copper mug')
london = pd.read_csv("data/london_transactions.csv.gz", compression="gzip", sep=r"\t", names=['datetime','drink','price'], index_col=0)
budapest = pd.read_csv("data/budapest.csv.gz", compression="gzip",names=['datetime','drink','price'], index_col=0, header=0)
ny = pd.read_csv("data/ny.csv.gz", compression="gzip",names=['datetime','drink','price'], index_col=0, header=0)

C:\Users\alast\AppData\Local\Temp\ipykernel_1004\2707261424.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  london = pd.read_csv("data/london_transactions.csv.gz", compression="gzip", sep=r"\t", names=['datetime','drink','price'], index_col=0)


In [33]:
london['datetime'] = london['datetime'].apply(pd.to_datetime)
london['BarID'] = 1
budapest['datetime'] = budapest['datetime'].apply(pd.to_datetime)
budapest['BarID'] = 2
ny['datetime'] = ny['datetime'].apply(pd.to_datetime)
ny['BarID'] = 3

In [34]:
london.head(5)

,datetime,drink,price,BarID
0,2020-12-30 13:17:00,Mojito,5.5,1
1,2020-12-27 11:06:15,Sidecar,11.0,1
2,2020-12-25 17:40:36,Snowball,11.0,1
3,2020-12-28 08:55:02,Limeade,3.5,1
4,2020-12-30 20:10:03,Limeade,3.5,1


In [5]:
london['drink'].value_counts()

drink
Limeade                       3318
Snowball                      2991
Orangeade                     1831
Bellini                       1786
Raspberry Cooler              1566
                              ... 
Melya                           16
Penicillin                      15
Lemon Elderflower Spritzer      14
Popped cherry                    3
Royal Flush                      1
Name: count, Length: 230, dtype: int64

In [6]:
london.shape

(50000, 3)

In [7]:
budapest.head(5)

,datetime,drink,price
0,2020-12-30 15:15:53,Sweet Sangria,4.0
1,2020-12-31 18:02:26,Grim Reaper,4.0
2,2020-12-30 00:01:27,Kool-Aid Slammer,3.0
3,2020-12-26 21:58:30,Margarita,7.5
4,2020-12-27 20:51:36,Margarita,7.5


In [8]:
budapest['drink'].value_counts()

drink
Hawaiian Cocktail    2592
Imperial Cocktail    2511
Margarita            2453
Long Island Tea      1784
Manhattan            1397
                     ... 
Melya                  10
Penicillin              9
Royal Flush             6
Whiskey Sour            5
Popped cherry           2
Name: count, Length: 232, dtype: int64

In [9]:
budapest.shape

(50000, 3)

In [10]:
ny.head(5)

,datetime,drink,price
0,2020-12-26 22:47:00,Paradise,4.20
1,2020-12-26 08:34:00,Dry Martini,8.00
2,2020-12-31 18:41:00,Kill the cold Smoothie,11.00
3,2020-12-29 15:38:00,Cuba Libra,2.99
4,2020-12-26 07:09:00,Campari Beer,7.50


In [11]:
ny['drink'].value_counts()

drink
Cuba Libra       2322
Chicago Fizz     2232
Pina Colada      1832
Dry Martini      1826
Kurant Tea        417
                 ... 
Queen Bee          13
Caipirissima       13
Whiskey Sour        5
Royal Flush         3
Popped cherry       1
Name: count, Length: 231, dtype: int64

In [12]:
ny.shape

(50000, 3)

Need to take care around column names, separators and indexes as well as the format of the date time columns

Need a dimension table for the glass types, bar id and drinks combo bar as price may be different for a drink across the bars

In [13]:
cocktail_client = CocktailClient()

In [14]:
lst = cocktail_client.list_all_cocktails("a").json()

In [15]:
lst['drinks'][0]

{'idDrink': '17222',
 'strDrink': 'A1',
 'strDrinkAlternate': None,
 'strTags': None,
 'strVideo': None,
 'strCategory': 'Cocktail',
 'strIBA': None,
 'strAlcoholic': 'Alcoholic',
 'strGlass': 'Cocktail glass',
 'strInstructions': 'Pour all ingredients into a cocktail shaker, mix and serve over ice into a chilled glass.',
 'strInstructionsES': 'Vierta todos los ingredientes en una coctelera, mezcle y sirva con hielo en un vaso frío.',
 'strInstructionsDE': 'Alle Zutaten in einen Cocktailshaker geben, mischen und über Eis in ein gekühltes Glas servieren.',
 'strInstructionsFR': None,
 'strInstructionsIT': 'Versare tutti gli ingredienti in uno shaker, mescolare e servire con ghiaccio in un bicchiere freddo.',
 'strInstructionsZH-HANS': None,
 'strInstructionsZH-HANT': None,
 'strDrinkThumb': 'https://www.thecocktaildb.com/images/media/drink/2x8thr1504816928.jpg',
 'strIngredient1': 'Gin',
 'strIngredient2': 'Grand Marnier',
 'strIngredient3': 'Lemon Juice',
 'strIngredient4': 'Grenadine'

In [16]:
marg = cocktail_client.get_info_for_cocktail_name("margarita")

In [17]:
marg.json()

{'drinks': [{'idDrink': '11007',
   'strDrink': 'Margarita',
   'strDrinkAlternate': None,
   'strTags': 'IBA,ContemporaryClassic',
   'strVideo': None,
   'strCategory': 'Ordinary Drink',
   'strIBA': 'Contemporary Classics',
   'strAlcoholic': 'Alcoholic',
   'strGlass': 'Cocktail glass',
   'strInstructions': 'Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.',
   'strInstructionsES': None,
   'strInstructionsDE': 'Reiben Sie den Rand des Glases mit der Limettenscheibe, damit das Salz daran haftet. Achten Sie darauf, dass nur der äußere Rand angefeuchtet wird und streuen Sie das Salz darauf. Das Salz sollte sich auf den Lippen des Genießers befinden und niemals in den Cocktail einmischen. Die anderen Zutaten mit Eis schütteln und vorsi

In [18]:
len(marg.json()['drinks'])

6

In [19]:
for i in marg.json()['drinks']:
    print (i['strDrink'])

Margarita
Blue Margarita
Tommy's Margarita
Whitecap Margarita
Strawberry Margarita
Smashed Watermelon Margarita


#Starting dim transforms

In [20]:
glasses = bar_data.groupby("glass_type").max().reset_index()
glasses['ID'] = glasses.index
glasses.rename(columns = {'ID': 'GlassID', 'glass_type': 'GlassName'}, inplace=True)

In [22]:
glasses.head(5)

,GlassName,stock,bar,GlassID
0,balloon glass,48,new york,0
1,beer glass,48,new york,1
2,beer mug,42,new york,2
3,beer pilsner,48,new york,3
4,brandy snifter,48,new york,4


In [23]:
def return_glass(cocktail_client, drink_name):
    res = cocktail_client.get_info_for_cocktail_name(drink_name).json()
    return (res['drinks'][0]['strGlass'])

In [24]:
all_drinks = pd.DataFrame(set(london['drink'].unique()) | set(budapest['drink'].unique()) | set(ny['drink'].unique()), columns=["DrinkName"])
all_drinks['DrinkID'] = all_drinks.index
all_drinks["Glass"] = all_drinks.apply(lambda row: return_glass(cocktail_client, row['DrinkName']), axis=1)
all_drinks['Glass'] = all_drinks['Glass'].apply(lambda row: row.lower())
drinks_glass = pd.merge(all_drinks, glasses[['GlassID','GlassName']], left_on='Glass', right_on='GlassName')

In [25]:
all_drinks.head(5)

,DrinkName,DrinkID,Glass
0,Turf Cocktail,0,cocktail glass
1,Zizi Coin-coin,1,margarita/coupette glass
2,Jewel Of The Nile,2,cocktail glass
3,Irish Spring,3,collins glass
4,Royal Flush,4,old-fashioned glass


In [26]:
drinks_glass.head(5)

,DrinkName,DrinkID,Glass,GlassID,GlassName
0,Turf Cocktail,0,cocktail glass,6,cocktail glass
1,Jewel Of The Nile,2,cocktail glass,6,cocktail glass
2,Allegheny,11,cocktail glass,6,cocktail glass
3,Kentucky Colonel,17,cocktail glass,6,cocktail glass
4,Dirty Martini,22,cocktail glass,6,cocktail glass


In [27]:
bars = pd.DataFrame({"BarID": [1,2,3],"BarName": ["london","budapest","new york"]})
bars

,BarID,BarName
0,1,london
1,2,budapest
2,3,new york


In [35]:
london_price = london[['drink','price','BarID']].groupby("drink").first()
london_price.reset_index(inplace=True)

budapest_price = budapest[['drink','price','BarID']].groupby("drink").first()
budapest_price.reset_index(inplace= True)

ny_price = ny[['drink','price','BarID']].groupby("drink").first()
ny_price.reset_index(inplace=True)

price_frames = [london_price, budapest_price, ny_price]
all_prices= pd.concat(price_frames)
prices_drinks = pd.merge(all_prices, all_drinks, left_on="drink", right_on = "DrinkName")

In [36]:
all_prices.head(5)

,drink,price,BarID
0,A. J.,11.0,1
1,Ace,12.0,1
2,Adam,3.5,1
3,Addison,12.0,1
4,Affair,8.0,1


In [37]:
prices_drinks.head(5)

,drink,price,BarID,DrinkName,DrinkID,Glass
0,A. J.,11.0,1,A. J.,165,cocktail glass
1,A. J.,11.0,2,A. J.,165,cocktail glass
2,A. J.,11.0,3,A. J.,165,cocktail glass
3,Ace,12.0,1,Ace,29,martini glass
4,Ace,12.0,2,Ace,29,martini glass


In [38]:
sales_frames = [london, budapest, ny]
all_sales = pd.concat(sales_frames)
sales_drinks = pd.merge(all_sales, all_drinks, left_on='drink', right_on='DrinkName')

In [39]:
all_sales.head(5)

,datetime,drink,price,BarID
0,2020-12-30 13:17:00,Mojito,5.5,1
1,2020-12-27 11:06:15,Sidecar,11.0,1
2,2020-12-25 17:40:36,Snowball,11.0,1
3,2020-12-28 08:55:02,Limeade,3.5,1
4,2020-12-30 20:10:03,Limeade,3.5,1


In [40]:
sales_drinks.head(5)

,datetime,drink,price,BarID,DrinkName,DrinkID,Glass
0,2020-12-30 13:17:00,Mojito,5.5,1,Mojito,32,highball glass
1,2020-12-26 10:01:40,Mojito,5.5,1,Mojito,32,highball glass
2,2020-12-29 13:27:20,Mojito,5.5,1,Mojito,32,highball glass
3,2020-12-29 19:11:57,Mojito,5.5,1,Mojito,32,highball glass
4,2020-12-31 06:33:56,Mojito,5.5,1,Mojito,32,highball glass


In [44]:
bar_stock = pd.merge(bar_data, bars, left_on='bar', right_on='BarName')
bar_glasses_stock = pd.merge(bar_stock, glasses[['GlassName','GlassID']], left_on="glass_type", right_on='GlassName')

In [48]:
bar_glasses_stock.head(5)

,glass_type,stock,bar,BarID,BarName,GlassName,GlassID
0,cocktail glass,8,budapest,2,budapest,cocktail glass,6
1,cocktail glass,42,london,1,london,cocktail glass,6
2,cocktail glass,211,new york,3,new york,cocktail glass,6
3,shot glass,31,budapest,2,budapest,shot glass,26
4,shot glass,33,london,1,london,shot glass,26


In [7]:
con = sqlite3.connect("bars.db")
query = """SELEct * FROM drinks_dim"""
df = pd.read_sql(query, con)

In [8]:
df

,DrinkID,DrinkName,GlassID
0,0,Chicago Fizz,12
1,2,Snowball,12
2,7,Broadside,12
3,9,Gin Fizz,12
4,10,Mojito,12
...,...,...,...
233,232,Irish Coffee,14
234,182,Mango Mojito,15
235,202,Oatmeal Cookie,19
236,204,Limona Corona,1
